### <font color = yellow> This is a clean copy of fifo_rows_redo.ipynb at creation point

In [1]:
library(tibble)

In [2]:
cat("One", "Two", "Three", sep = " | ", fill = TRUE)

One | Two | Three


In [3]:
options(pillar.width = Inf)


## <font color = yellow> create_df function () - reads .csv

In [4]:
   
df <- read.csv("fifo_new.csv", header = TRUE, sep = ",")
df$date <- as.Date(df$date, origin = "1899-12-30")

In [5]:
head(df,2)

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,buy_pps,buy_basis,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>
1,1,2022-12-31,Buy,AMC,1050,4.0700,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [6]:
df$true_false <- FALSE
df$flag <- FALSE

In [7]:
head(df, 2)

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>
1,1,2022-12-31,Buy,AMC,1050,4.0700,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE


## <font color = yellow>1. Create data & dfs

In [8]:
#    df_records <- data.frame()
sale_records <- data.frame() 


In [9]:
# create dfs from filtered df
buys <- df[df$type == 'Buy', ]  # Filter buys
sells <- df[df$type == 'Sell', ]  # Filter sells
b <- buys[1,]
s <- sells[1,]
b

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>
1,1,2022-12-31,Buy,AMC,1050,4.07,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE


## <font color = yellow> Examining data and dfs

In [10]:
nrow(buys)
nrow(sells)
dim(sells)
head(buys, 2)
head(sells,2)

[1] 23

[1] 18

[1] 18 25

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>
1,1,2022-12-31,Buy,AMC,1050,4.070,4273.5,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE
3,3,2023-01-20,Buy,AMC,1000,5.625,5625.0,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE


,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>
2,2,2023-01-27,Sell,AMC,1000,5.51240,5512.40,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE
4,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE


In [11]:
dim(b)
str(b)

[1]  1 25

'data.frame':	1 obs. of  25 variables:
 $ index        : int 1
 $ date         : Date, format: "2022-12-31"
 $ type         : chr "Buy"
 $ co           : chr "AMC"
 $ quan         : int 1050
 $ pps          : num 4.07
 $ basis        : num 4274
 $ sale_quan    : int 0
 $ tally        : int 0
 $ cf_quan      : int 0
 $ cf_pps       : num 0
 $ cf_basis     : num 0
 $ buy_quan     : int 0
 $ buy_pps      : num 0
 $ buy_basis    : num 0
 $ sell_quan    : int 0
 $ sell_pps     : num 0
 $ sell_basis   : num 0
 $ profit_loss  : num 0
 $ Sell_cf      : int 0
 $ Sell_cf_pps  : num 0
 $ Sell_cf_basis: num 0
 $ TorF         : int 0
 $ true_false   : logi FALSE
 $ flag         : logi FALSE


## <font color = yellow>2. Update b - the variable that records each buy/sell transaction (w/in loop)
    
#### b>s, b<s, b==s,

In [12]:
for(i in 1:3) {
    cat("for loop, loop is i: ", i, "\n")
    }

for loop, loop is i:  1 
for loop, loop is i:  2 
for loop, loop is i:  3 


#### <font color = lime> Basic version: when b == s ( creates no c/f)
    
#### <font color = lime> Basic version: when b > s (creates b c/f)    
    
#### <font color = lime> Basic version: when b < s (creates s c/f)
    

In [13]:
# # update b based upon the Sale Type: b>s, b<s, b==s 

update_b <- function() {


        #  b > s                       Buy C/F 
        if ( b$quan > s$quan) { 
            
                        
            b$sale_quan  <- s$quan     # sb the sell quantity
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$bot_cf_quan <- (b$quan - s$quan)        
            b$bot_cf_pps <- b$pps
            b$bot_cf_basis <- b$bot_cf_quan * b$pps           

            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has buy c/f 
            b$flag <- TRUE

           # s$type <- "Buy-CF"
    
       # ( s > b ), has sell carryforward
 # cat("If s > b, then S_cf_quan is: ", S_cf_quan, " and Sell_cf is ", b$Sell_cf, "\n")  
        }else if(b$quan < s$quan) {
            
            b$basis <-  b$quan * b$pps
            b$sale_quan <- b$quan
           
            # b$buy_quan <- s$quan 
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$basis
          

        
            # Sell c/f
            sold_cf_quan <- s$quan - b$quan           
            b$sold_cf_quan <- sold_cf_quan
            b$sold_cf_pps <- s$pps
            b$sold_cf_basis <- sold_cf_quan * s$pps
            

            # b$sell_quan <- s$quan    # cannot sell more than the buy_quan
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            
            # Has sell c/f 
            b$true_false <- TRUE
            
            
            # b$sale <- sale

            s$type <- "SELL-CF"   
            
        }else if (s$quan == b$quan) {
            
            
             # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <- b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "SOLD"
            
            }
            
        return(b)   
}  # fct

updated_b <- update_b()
updated_b  
    

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag,bot_cf_quan,bot_cf_pps,bot_cf_basis
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>,<int>,<dbl>,<dbl>
1,1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,0,0,⋯,1442.4,0,0,0,0,FALSE,TRUE,50,4.07,203.5


In [14]:
# print(t(updated_b))

In [15]:
# updated_b <- update_b()
# updated_b

### <font color = lime> Works with each sale type:( b > s , s > b, b = s, buys_cf, sells_cf )

### <font color = yellow>3. Create a new variable from buys/sells row 1, then delete row 1 from buys and sells

In [16]:
#  save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]

nrow(buys)
nrow(sells)

b1 <- buys [1, ]
s1 <- sells [1, ]
dim(s1)
# save_buy_one <- buys[-1,]

buys <- buys[-1, ]
sells <- sells[-1, ]

nrow(buys)
nrow(sells)

[1] 23

[1] 18

[1]  1 25

[1] 22

[1] 17

In [17]:
sells[1,]

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>
4,4,2023-02-03,Sell,AMC,1000,6.68255,6682.55,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE


In [18]:
s1

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,sell_quan,sell_pps,sell_basis,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<int>,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>
2,2,2023-01-27,Sell,AMC,1000,5.5124,5512.4,0,0,0,⋯,0,0,0,0,0,0,0,0,FALSE,FALSE


### <font color = yellow>4. Update the sales_records (rbind) with updated_b for each transaction (loop)

In [19]:
sale_records <- rbind(sale_records, updated_b)
sale_records
                      

index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag,bot_cf_quan,bot_cf_pps,bot_cf_basis
<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>,<int>,<dbl>,<dbl>
1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,0,0,⋯,1442.4,0,0,0,0,FALSE,TRUE,50,4.07,203.5


### <font color = yellow> 5. Update buys or sells with a new row (add_row) - whenever a transaction (loop) includes  c/f
    
###### Uses add_row () to transform any existing carryforward (buy or sell type) into the next element that will be processed for next (loop) buy/sell transaction


In [20]:
# good example from R:  ?add_row
df <- tibble(x = 1:3, y = 3:1)

df <- df %>% add_row(x = 4, y = 0, .before = 2)
df

x,y
<dbl>,<dbl>
1,3
4,0
2,2
3,1


#### <font color = lime> How it's done

In [21]:
last_row <- tail(sale_records, n = 1)
last_row

,index,date,type,co,quan,pps,basis,sale_quan,tally,cf_quan,⋯,profit_loss,Sell_cf,Sell_cf_pps,Sell_cf_basis,TorF,true_false,flag,bot_cf_quan,bot_cf_pps,bot_cf_basis
,<int>,<date>,<chr>,<chr>,<int>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<int>,<dbl>,<dbl>,<int>,<lgl>,<lgl>,<int>,<dbl>,<dbl>
1,1,2022-12-31,Buy,AMC,1050,4.07,4273.5,1000,0,0,⋯,1442.4,0,0,0,0,FALSE,TRUE,50,4.07,203.5


In [22]:
# Evaluate the type of carryforward, then update buys or sells w/cf to prepare for next transaction (loop)

update_withcf <- function() {

    last_row <- tail(sale_records, n = 1)

    if (last_row$flag == TRUE && last_row$true_false == FALSE ) {
        cat("\n", "There is Buy carryforward. Update buys ", last_row$bot_cf_quan, "\n")

        b1 <- last_row

        b1$type <- "Buy CF"
        b1$quan <- b1$bot_cf_quan
        b1$pps <- b1$bot_cf_pps
        b1$basis <- b1$bot_cf_basis
        b1$sale_quan <- 0
        b1[ , 10:ncol(b1)] <- 0

        test <- buys
        test <- test %>% add_row(b1, .before = 1)
        buys <- test



    } else if (last_row$flag == FALSE && last_row$true_false == TRUE ) {
        cat("\n", "There is Sold carryforward. Update sells ", last_row$sold_cf_quan, "\n")
            cat("dim(s1) : ", dim(s1), "\n")

        s1$type <- "Sell-CF"
        s1$quan <- updated_b$sold_cf_quan
        s1$pps <-  updated_b$sold_cf_pps
        s1$basis <- updated_b$sold_basis
        s1$buy_quan <- 0
        s1$buy_pps <- 0
        s1$buy_basis <- 0
        s1$sell_quan <- 0
        s1$sell_pps <- 0 
        s1$sell_basis <- 0
        s1$profit_loss <- 0    

        # s1$true_false <- TRUE

         test <- sells
         test <- test %>% add_row(s1, .before = 1)
         sells <- test    
        }

        if(last_row$flag == TRUE) {
            cat("printing updated buys[1 ] w/ bot c/f ", "\n")
            b1
            #buys[1, ]
        }else {
            cat("printing: s1 ... Sells c/f ", "\n")
            s1
            #sells[1, ]
        }
    }  # fct

update_withcf()

updated_b



 There is Buy carryforward. Update buys  50 


ERROR: [1m[33mError[39m in `add_row()`:[22m
[1m[22m[33m![39m New rows can't add columns.
[31m✖[39m Can't find columns `bot_cf_quan`, `bot_cf_pps`, and
  `bot_cf_basis` in `.data`.


#### <font color = lime> QC: review buys-sells for updated carryforward
    
#### When c/f exists, that c/f (type) must be inserted into the next row of either buys or sells

In [ ]:

buys[1, ]
sells[1,]

#### <font color = lime> QC: review sale_record for accuracy

In [ ]:
sale_records

### <font color = lime> Got it!  The components above are modular and can be combined into a loop

In [ ]:
stopps #  Old code

## <font color = yellow> Continue! Inputs: buys, sells, sale_records 

In [ ]:
nrow(buys)    # if  b > s, then -1, +1 , t/f 23/17
nrow(sells)   # if  s > b, then -1, +1 , t/f 22/18
nrow(sale_records)

# 

## <font color = yellow> 1.  b and s

In [ ]:
b <- buys[1,]
s <- sells[1,]
b
s

In [ ]:
sale_records


## <font color = yellow> 2.  updated_b  <-  b

In [ ]:
# update b based on Sale Type: 

update_b <- function(b) {
    
     cat("Inside update_b, test for: $quan < s$quan && b$type == 'Buy CF': ", "\n")
    
        if(b$quan < s$quan && b$type == 'Buy CF') {
            cat("True", "\n")
            }

        sale <- s$quan
        bs_ratio <- b$quan / s$quan
        bot_cf_quan <-  b$quan - s$quan

        # assign the cf values - if they exist
         #  b > s ... restricted to Buy only
        if ( b$quan > s$quan && b$type == "Buy") {  
            
            # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$bot_cf_quan <- (b$quan - s$quan)        
            b$cf_pps <- b$pps
            b$cf_basis <- bot_cf_quan * b$pps           

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "Buy-CF"           
        
            
        }else if( b$quan < s$quan && b$type == 'Buy CF') {    

                # protect b from errors
            temp <- b  

            temp$type <- "Buy-2xCF"           
            temp$sell_quan <- temp$quan
            temp$sell_pps <- sale_records$sell_pps[1]
            temp$sell_basis <- temp$sell_quan * sale_records$sell_pps
            temp$profit_loss <- temp$sell_basis - temp$basis

            temp$Sell_cf  <-  sale_records$sell_quan - sale_records$bot_cf_quan 
            temp$Sell_cf_pps   <- sale_records$sell_pps
            # update b with temp
            b <- temp
            s$type <- "Buy-2xCF"       


               # ( s > b ), has sell carryforward
        }else if(s$quan > b$quan) {            
            
            b$sale_quan <- b$quan
           # b$buy_quan <- s$quan 
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps
        
            # Sell c/f
            S_cf_quan <- s$quan - b$quan           
            b$Sell_cf <-S_cf_quan
            b$Sell_cf_pps <- s$pps
            b$Sell_cf_basis <- S_cf_quan * s$pps
            
     
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis          
            s$type <- "SELL-CF"   
            
        }else if (s$quan == b$quan) {            
           
             # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
            b$sale <- sale1
            s$type <- "SOLD"
            
            }
    
        return(b)   
}  # fct

updated_b <- update_b(b)
updated_b  
    

In [ ]:
print(t(updated_b))

## <font color = yellow>3.  buys <- buy[-1, ], sells <- sells[-1, ]
    
#### Nec to start loop at the first (unprocessed rows in buys / sells at the top of the loop

In [ ]:
save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]

nrow(buys)

save_sell_one <- sells [1, ]
save_buy_one <- buys[-1,]

buys <- buys[-1, ]
sells <- sells[-1, ]

nrow(buys)

## <font color = yellow> 4. sale_records <- rbind(sale_records, updated_b)

In [ ]:
sale_records <- rbind(sale_records, updated_b)
sale_records

In [ ]:
# stopps

# <font color = yellow> Looks good.  Loop 3   
    

In [ ]:
stopps

### <font color = lime > 1.  b and s

In [ ]:
b <- buys[1,]
s <- sells[1,]
b
s

### <font color = lime> 2.  updated_b  <-  b

In [ ]:
# update b based on Sale Type: 

update_b <- function(b) {
    
     cat("Inside update_b, test for: $quan < s$quan && b$type == 'Buy CF': ", "\n")
    
        if(b$quan < s$quan && b$type == 'Buy CF') {
            cat("True", "\n")
            }

        sale <- s$quan
        bs_ratio <- b$quan / s$quan
        bot_cf_quan <-  b$quan - s$quan

        # assign the cf values - if they exist
         #  b > s ... restricted to Buy only
        if ( b$quan > s$quan && b$type == "Buy") {  
            
            # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps


            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$bot_cf_quan <- (b$quan - s$quan)        
            b$cf_pps <- b$pps
            b$cf_basis <- cf_quan * b$pps           

            b$profit_loss <- b$sell_basis - b$buy_basis
            # b$sale <- sale

            s$type <- "Buy-CF"           
        
            
        }else if( b$quan < s$quan && b$type == 'Buy CF') {    

                # protect b from errors
            temp <- b  

            temp$type <- "Buy-2xCF"           
            temp$sell_quan <- temp$quan
            temp$sell_pps <- sale_records$sell_pps[1]
            temp$sell_basis <- temp$sell_quan * sale_records$sell_pps
            temp$profit_loss <- temp$sell_basis - temp$basis

            temp$Sell_cf  <-  sale_records$sell_quan - sale_records$bot_cf_quan 
            temp$Sell_cf_pps   <- sale_records$sell_pps
            # update b with temp
            b <- temp
            s$type <- "Buy-2xCF"       


               # ( s > b ), has sell carryforward
        }else if(s$quan > b$quan) {            
            
            b$sale_quan <- b$quan
           # b$buy_quan <- s$quan 
            b$buy_quan <- b$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps
        
            # Sell c/f
            S_cf_quan <- s$quan - b$quan           
            b$Sell_cf <-S_cf_quan
            b$Sell_cf_pps <- s$pps
            b$Sell_cf_basis <- S_cf_quan * s$pps
            
     
            b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- b$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis          
            s$type <- "SELL-CF"   
            
        }else if (s$quan == b$quan) {            
           
             # b$sale_quan <- b$quan
            b$buy_quan <- s$quan
            b$buy_pps <-  b$pps
            b$buy_basis <-  b$buy_quan * b$pps

            b$sell_quan <- s$quan    # cannot sell more than the buy_quan
           # b$sell_quan <- b$quan
            b$sell_pps <- s$pps
            b$sell_basis <- s$quan * s$pps

            b$profit_loss <- b$sell_basis - b$buy_basis
           # b$sale <- "sale1"
            s$type <- "SOLD"
            
            }
    
        return(b)   
}  # fct

updated_b <- update_b(b)
updated_b  
    

### <font color = lime>3.  buys <- buy[-1, ], sells <- sells[-1, ]
    
##### Nec to start loop at the first (unprocessed rows in buys / sells at the top of the loop

In [ ]:
save_sell_one <- data.frame()   #  keep for c/f >> Sell[1, ]

nrow(buys)

save_sell_one <- sells [1, ]
save_buy_one <- buys[-1,]

buys <- buys[-1, ]
sells <- sells[-1, ]

# nrow(buys)

### <font color = lime> 4. sale_records <- rbind(sale_records, updated_b)

In [ ]:
sale_records <- rbind(sale_records, updated_b)
sale_records